In [34]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
from cartopy.io.shapereader import Reader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import (OCEAN, LAKES, BORDERS, COASTLINE, RIVERS, COLORS,
                             LAND)
import netCDF4

EDGAR_ds = xr.open_dataset(r'F:/edgar_emis/EDGARHTAP2_MEIC2015_CH4_2010.0.1x0.1.nc')

EDGAR_ds = EDGAR_ds.sel(time=slice("2010-01-01", "2010-12-16")) #create a slice of EDGAR data for 2010. Use the same dates as CMIP
#time co-ordinate in EDGAR file looks weird with 12 data points on 01/01/2010 instead of different data points, but as far as I can tell, this is correct
EDGAR_ds = EDGAR_ds.sel(lon = slice(-15,21), lat=slice(30,61))
print (EDGAR_ds)

#this will work out the proportion that each agricultural subsector is of the total sector included in the emissions.

#4A is enteric fermentation
E_4A_ch4_data = EDGAR_ds['emis_4A'] #choose the sector you want to isolate
E_4A_ch4_data = E_4A_ch4_data.mean(dim='lat')
E_4A_ch4_data = E_4A_ch4_data.mean(dim='lon')
E_4A_ch4_data = E_4A_ch4_data.mean(dim='time') #should give me a mean value for all of the domain

#4B is manure management

E_4B_ch4_data = EDGAR_ds['emis_4B'] 
E_4B_ch4_data = E_4B_ch4_data.mean(dim='lat')
E_4B_ch4_data = E_4B_ch4_data.mean(dim='lon')
E_4B_ch4_data = E_4B_ch4_data.mean(dim='time')

#4C_4D is agricultural soils

E_4CD_ch4_data = EDGAR_ds['emis_4C_4D'] 
E_4CD_ch4_data = E_4CD_ch4_data.mean(dim='lat')
E_4CD_ch4_data = E_4CD_ch4_data.mean(dim='lon')
E_4CD_ch4_data = E_4CD_ch4_data.mean(dim='time')

#4F is agricultural fires

E_4F_ch4_data = EDGAR_ds['emis_4F'] 
E_4F_ch4_data = E_4F_ch4_data.mean(dim='lat')
E_4F_ch4_data = E_4F_ch4_data.mean(dim='lon')
E_4F_ch4_data = E_4F_ch4_data.mean(dim='time')

#print(E_4F_ch4_data)
#work out total

total_agriculture = E_4A_ch4_data + E_4B_ch4_data + E_4CD_ch4_data + E_4F_ch4_data

print ("total agriculture emissions =", total_agriculture)

#percentages for each sub-sector

percent_4A = (E_4A_ch4_data/total_agriculture)*100
print (r"percentage in 4A (enteric fermentation) is", percent_4A, "%")
percent_4B = (E_4B_ch4_data/total_agriculture)*100
print (r"percentage in 4B (manure management) is", percent_4B, "%")
percent_4CD = (E_4CD_ch4_data/total_agriculture)*100
print (r"percentage in 4CD (agricultural soils) is", percent_4CD, "%")
percent_4F = (E_4F_ch4_data/total_agriculture)*100
print (r"percentage in 4F (agricultural fires) is", percent_4F, "%")

#use these percentages to calculate these sectors from CMIP6 agriculture sector

#power sector

power_ds = xr.open_dataset('F:/edgar_emis/v6.0_CH4_2010_ENE.0.1x0.1.nc') #This is annual means of 1A1
print (power_ds)
power_CH4_emis = power_ds.mean(dim='lat')
power_CH4_emis = power_CH4_emis.mean(dim='lon')  

#also need fossil fuel exploitation sectors. Can get these from EDGAR as already there.

E_1B1_ch4_data = EDGAR_ds['emis_1B1'] #this is coal exploitation
E_1B1_ch4_data = E_1B1_ch4_data.mean(dim='lat')
E_1B1_ch4_data = E_1B1_ch4_data.mean(dim='lon')
E_1B1_ch4_data = E_1B1_ch4_data.mean(dim='time')

E_1B2a_ch4_data = EDGAR_ds['emis_1B2a'] #this is oil exploitation
E_1B2a_ch4_data = E_1B2a_ch4_data.mean(dim='lat')
E_1B2a_ch4_data = E_1B2a_ch4_data.mean(dim='lon')
E_1B2a_ch4_data = E_1B2a_ch4_data.mean(dim='time')

E_1B2b_ch4_data = EDGAR_ds['emis_1B2b'] #this is gas exploitation
E_1B2b_ch4_data = E_1B2b_ch4_data.mean(dim='lat')
E_1B2b_ch4_data = E_1B2b_ch4_data.mean(dim='lon')
E_1B2b_ch4_data = E_1B2b_ch4_data.mean(dim='time')

power_emis_total = power_CH4_emis + E_1B1_ch4_data + E_1B2a_ch4_data + E_1B2b_ch4_data


percent_1A1 = (power_CH4_emis/power_emis_total)*100
print (r'percentage in 1A1 (combustion for power) is', percent_1A1, '%')
percent_1B1 = (E_1B1_ch4_data/power_emis_total)*100
print (r'percentage in 1B1 (coal exploitation) is', percent_1B1, '%')
percent_1B2a = (E_1B2a_ch4_data/power_emis_total)*100
print (r'percentage in 1B2a (oil exploitation) is', percent_1B2a, '%')
percent_1B2b = (E_1B2b_ch4_data/power_emis_total)*100
print (r'percentage in 1B2b (gas exploitation) is', percent_1B2b, '%')

#not sure this is correct - percentage of combustion for power seems extremely low in comparison to fuel extraction

#industrial processes and combustion for manufacturing are combined in EDGAR-HTAP2, but can read in individual datasets with just the annual mean already calculated so no time dimension.
#this is fine because I just want the percentage difference anyway


E_2_ch4_data =  EDGAR_ds['emis_2'] #this is the industrial sector in my edgar emissions, already selected by Europe and year. Process as normal
E_2_ch4_data = E_2_ch4_data.mean(dim='lat')
E_2_ch4_data = E_2_ch4_data.mean(dim='lon')
E_2_ch4_data = E_2_ch4_data.mean(dim='time')

indfires_ds = xr.open_dataset (r'F:/edgar_emis/v6.0_CH4_2010_IND.0.1x0.1.nc')
indfires_ds = indfires_ds.sel(lon = slice(-15,21), lat=slice(30,61))
indfires_CH4_emis = indfires_ds['emi_ch4']
indfires_CH4_emis = indfires_CH4_emis.mean(dim='lat')
indfires_CH4_emis = indfires_CH4_emis.mean(dim='lon')



total_industrial = E_2_ch4_data + indfires_CH4_emis
percent_1A2 = (indfires_CH4_emis/total_industrial)*100
print (r'percentage in 1A2 (industrial fires) is', percent_1A2, '%') #54 percent is fires - is this correct?

#so would be routing 54% of CMIP6 "industry" sector to 1A1_1A2


#percentages for industrial
#percent_2 = (E_2_ch4_data/total_industrial)*100
#print (r"percentage in 2 (industrial processes) is", percent_2, "%")

#now do waste, 6A is landfills, 6B is incineration, 6C (grouped with 6A) is from liquid waste
E_6A_ch4_data = EDGAR_ds['emis_6A_6C']
E_6A_ch4_data = E_6A_ch4_data.mean(dim='lat')
E_6A_ch4_data = E_6A_ch4_data.mean(dim='lon')
E_6A_ch4_data = E_6A_ch4_data.mean(dim='time')

E_6B_ch4_data = EDGAR_ds['emis_6B']
E_6B_ch4_data = E_6B_ch4_data.mean(dim='lat')
E_6B_ch4_data = E_6B_ch4_data.mean(dim='lon')
E_6B_ch4_data = E_6B_ch4_data.mean(dim='time')

total_waste = E_6A_ch4_data + E_6B_ch4_data

percent_6A = (E_6A_ch4_data/total_waste)*100
print (r"percentage in 6A (landfills) is", percent_6A, "%")

percent_6B = (E_6B_ch4_data/total_waste)*100
print (r"percentage in 6B (incineration) is", percent_6B, "%")

#Transport & shipping sector - biggest thing will be combining the shipping & other land transport sectors for the EDGAR emissions
#need to know proportion of non-road land transport from all land transport.

non_road_ds = xr.open_dataset(r'F:/edgar_emis/v6.0_CH4_2010_TNR_Other.0.1x0.1.nc') #1A3c and 1A3e
non_road_CH4_emis = non_road_ds.mean(dim='lat')
non_road_CH4_emis = non_road_CH4_emis.mean(dim='lon')
road_ds = xr.open_dataset(r'F:/edgar_emis/v6.0_CH4_2010_TRO_noRES.0.1x0.1.nc') #this is "no resuspension" I don't know what that means, but there's no just road transport with it
road_CH4_emis = road_ds.mean(dim='lat')
road_CH4_emis = road_CH4_emis.mean(dim='lon')

total_land_CH4 = non_road_CH4_emis + road_CH4_emis

percent_1A3ce = (non_road_CH4_emis/total_land_CH4)*100
print (r"percentage in 1A3ce (non road land transport) is", percent_1A3ce, "%")
percent_1A3b = (road_CH4_emis/total_land_CH4)*100
print (r"percentage in 1A3ce (road transport) is", percent_1A3b, "%")

#so to create sector 1A3cde from CMIP6 emissions, add all of shipping and 1.245% of transport.

#aviation- only cruise altitude emissions in EDGAR for methane. Bear that in mind.



                            








<xarray.Dataset>
Dimensions:          (lat: 310, lon: 210, time: 12)
Coordinates:
  * lat              (lat) float32 30.05 30.15 30.25 30.35 ... 60.75 60.85 60.95
  * lon              (lon) float32 0.05 0.15 0.25 0.35 ... 20.75 20.85 20.95
  * time             (time) datetime64[ns] 2010-01-01 2010-01-01 ... 2010-01-01
Data variables:
    emis_tot         (time, lat, lon) float32 ...
    date             (time) int32 ...
    datesec          (time) int32 ...
    emis_1A1_1A2     (time, lat, lon) float32 ...
    emis_1A3a_c_d_e  (time, lat, lon) float32 ...
    emis_1A3b        (time, lat, lon) float32 ...
    emis_1A4         (time, lat, lon) float32 ...
    emis_1B1         (time, lat, lon) float32 ...
    emis_1B2a        (time, lat, lon) float32 ...
    emis_1B2b        (time, lat, lon) float32 ...
    emis_2           (time, lat, lon) float32 ...
    emis_4A          (time, lat, lon) float32 ...
    emis_4B          (time, lat, lon) float32 ...
    emis_4C_4D       (time, lat, lon) 